# UTM Source Attribution Analysis

**Objective:** Analyze which acquisition sources drive high-quality signups and user engagement.

**Key Question:** Which acquisition sources bring in users who stay engaged, and which sources are just noise?

**Analysis Components:**
1. Data loading and validation
2. dlt pipeline to DuckDB
3. Acquisition, engagement, and retention metrics
4. Quality scoring and segmentation
5. Business insights and recommendations

## Step 1: Configuration and Setup

Define analysis parameters and install required packages.

In [2]:
# Install required packages
!pip install dlt[duckdb] pandas matplotlib seaborn plotly scipy scikit-learn

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Analysis configuration parameters
CONFIG = {
    # Quality score weights
    'volume_weight': 0.2,
    'engagement_weight': 0.4,
    'retention_weight': 0.4,
    
    # Quality tier thresholds
    'high_quality_threshold': 70,
    'medium_quality_threshold': 40,
    
    # Retention definitions
    'retention_min_events': 2,
    'retention_min_days': 7,
    'retention_month_days': 30,
    
    # Minimum sample size
    'min_users_for_analysis': 5,
    
    # File paths
    'input_file': 'WorkingStudentDataTask-dlthub.csv',
    'output_quality_file': 'utm_source_quality_analysis.csv',
    'output_summary_file': 'utm_attribution_summary.csv',
    'database_name': 'utm_attribution_pipeline.duckdb',
    'dataset_name': 'user_attribution'
}

print("Configuration loaded successfully")
print(f"Input file: {CONFIG['input_file']}")
print(f"Database: {CONFIG['database_name']}")
print(f"Quality score weights - Volume: {CONFIG['volume_weight']}, Engagement: {CONFIG['engagement_weight']}, Retention: {CONFIG['retention_weight']}")

Configuration loaded successfully
Input file: WorkingStudentDataTask-dlthub.csv
Database: utm_attribution_pipeline.duckdb
Quality score weights - Volume: 0.2, Engagement: 0.4, Retention: 0.4


## Step 2: Import Libraries

In [4]:
import dlt
import pandas as pd
import duckdb
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import warnings

warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully")

Libraries imported successfully


## Step 3: Load and Validate Data

In [5]:
# Load the CSV file
df = pd.read_csv(CONFIG['input_file'])

print(f"Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"\nColumns: {list(df.columns)}")
print(f"\nData preview:")
display(df.head())

Dataset loaded: 40345 rows, 12 columns

Columns: ['Contact ID', 'Field ID', 'Field Value', 'Contact Create date', 'Contact Update date', 'Fields Title', 'Fields Type', 'Fields Update date', 'Events Category', 'Events Create date', 'Events Hash', 'Events ID']

Data preview:


,Contact ID,Field ID,Field Value,Contact Create date,Contact Update date,Fields Title,Fields Type,Fields Update date,Events Category,Events Create date,Events Hash,Events ID
0,3414,1,NaN,"September 19, 2025, 2:59 AM","September 19, 2025, 2:59 AM",Agree to receive up to date dlthub communication,checkbox,"August 16, 2024, 1:00 PM",Education,"September 3, 2025, 1:34 PM",Q3A8R9T2,46.0
1,3425,1,NaN,"September 22, 2025, 6:56 PM","September 22, 2025, 6:56 PM",Agree to receive up to date dlthub communication,checkbox,"August 16, 2024, 1:00 PM",NaN,NaN,NaN,NaN
2,3384,1,NaN,"September 10, 2025, 2:27 AM","September 10, 2025, 2:27 AM",Agree to receive up to date dlthub communication,checkbox,"August 16, 2024, 1:00 PM",Education,"September 3, 2025, 1:34 PM",Q3A8R9T2,46.0
3,3382,1,||Yes||,"September 9, 2025, 11:52 AM","September 9, 2025, 11:52 AM",Agree to receive up to date dlthub communication,checkbox,"August 16, 2024, 1:00 PM",Education,"September 3, 2025, 1:34 PM",Q3A8R9T2,46.0
4,1909,5,dlt ELT Specialist,"March 21, 2025, 12:17 PM","March 21, 2025, 12:17 PM",Certificate Name,text,NaN,Education,"October 7, 2024, 3:17 PM",P6G1U8R2,22.0


In [6]:
# Data validation function
def validate_data(df):
    """Validate data quality and identify potential issues"""
    
    print("=" * 80)
    print("DATA VALIDATION REPORT")
    print("=" * 80)
    
    # Check for suspicious UTM sources
    utm_data = df[df['Fields Title'].str.contains('utm_source', case=False, na=False)]
    if len(utm_data) > 0:
        short_utms = utm_data[utm_data['Field Value'].str.len() < 3].shape[0]
        long_utms = utm_data[utm_data['Field Value'].str.len() > 50].shape[0]
        print(f"\nSuspicious UTM sources:")
        print(f"  - Very short (< 3 chars): {short_utms}")
        print(f"  - Very long (> 50 chars): {long_utms}")
    
    # Check for contacts with multiple UTM sources
    contact_utm_mapping = utm_data.groupby('Contact ID')['Field Value'].nunique()
    multi_utm = (contact_utm_mapping > 1).sum()
    print(f"\nContacts with multiple UTM sources: {multi_utm}")
    
    # Missing values
    print(f"\nMissing values:")
    print(f"  - Contact ID: {df['Contact ID'].isna().sum()}")
    print(f"  - Events ID: {df['Events ID'].isna().sum()}")
    
    # Duplicate records
    duplicates = df.duplicated().sum()
    print(f"\nDuplicate records: {duplicates}")
    
    # Event count distribution
    events_per_contact = df.groupby('Contact ID')['Events ID'].count()
    print(f"\nEvent count distribution:")
    print(f"  - Mean: {events_per_contact.mean():.2f}")
    print(f"  - Median: {events_per_contact.median():.2f}")
    print(f"  - Max: {events_per_contact.max()}")
    print(f"  - Contacts with >100 events: {(events_per_contact > 100).sum()}")
    
    print("\n" + "=" * 80)
    return df

# Run validation
df = validate_data(df)

DATA VALIDATION REPORT

Suspicious UTM sources:
  - Very short (< 3 chars): 0
  - Very long (> 50 chars): 0

Contacts with multiple UTM sources: 0

Missing values:
  - Contact ID: 0
  - Events ID: 72

Duplicate records: 15983

Event count distribution:
  - Mean: 15.12
  - Median: 7.00
  - Max: 162
  - Contacts with >100 events: 47



## Step 4: Data Cleaning and Preparation

In [7]:
df_clean = df.copy()

# Convert timestamp columns to datetime (handle out-of-bounds dates)
timestamp_cols = [col for col in df_clean.columns if 'date' in col.lower() or 'time' in col.lower()]
for col in timestamp_cols:
    try:
        df_clean[col] = pd.to_datetime(df_clean[col], errors='coerce')
        invalid_dates = df_clean[col].isna().sum()
        print(f"Converted {col} to datetime ({invalid_dates} invalid dates set to NaT)")
    except Exception as e:
        print(f"Could not convert {col}: {e}")

# Extract UTM source from Field Value where Fields Title contains 'utm_source'
print("\nExtracting UTM source data...")
utm_mapping = df_clean[df_clean['Fields Title'].str.contains('utm_source', case=False, na=False)][['Contact ID', 'Field Value']].copy()
utm_mapping.columns = ['Contact ID', 'utm_source']
utm_mapping = utm_mapping.dropna(subset=['utm_source'])
utm_mapping = utm_mapping.drop_duplicates(subset=['Contact ID'])

print(f"Extracted {len(utm_mapping)} contacts with UTM source")
print(f"Unique UTM sources: {utm_mapping['utm_source'].nunique()}")
print(f"\nUTM source distribution:")
print(utm_mapping['utm_source'].value_counts())

# Merge UTM source back to main dataframe
df_clean = df_clean.merge(utm_mapping, on='Contact ID', how='left')
df_clean['utm_source'] = df_clean['utm_source'].fillna('unknown')

# Standardize column names
if 'Events Category' in df_clean.columns:
    df_clean['event_category'] = df_clean['Events Category']
if 'Events ID' in df_clean.columns:
    df_clean['event_id'] = df_clean['Events ID']

print(f"\nCleaned dataset: {df_clean.shape}")
print("Data preparation complete")

Converted Contact Create date to datetime (0 invalid dates set to NaT)
Converted Contact Update date to datetime (0 invalid dates set to NaT)
Converted Fields Update date to datetime (16714 invalid dates set to NaT)
Converted Events Create date to datetime (736 invalid dates set to NaT)

Extracting UTM source data...
Extracted 663 contacts with UTM source
Unique UTM sources: 5

UTM source distribution:
utm_source
alexey_linkedin    403
dtc_newsletter     245
ActiveCampaign      12
adrian_test          2
chatgpt.com          1
Name: count, dtype: int64

Cleaned dataset: (40345, 15)
Data preparation complete


## Step 5: Defining dlt Resources and Pipeline
A star schema is implicitly created in DuckDB by the dlt pipeline, with contacts as the dimension and events as the fact table

In [8]:
@dlt.resource(name="contacts", write_disposition="replace")
def load_contacts():
    """Extract unique contacts with UTM source attribution"""
    contacts_df = df_clean[['Contact ID', 'utm_source']].drop_duplicates(subset=['Contact ID']).copy()
    contacts_df.columns = ['contact_id', 'utm_source']
    
    if 'Contact Create date' in df_clean.columns:
        first_seen = df_clean.groupby('Contact ID')['Contact Create date'].min().reset_index()
        first_seen.columns = ['contact_id', 'first_seen_at']
        contacts_df = contacts_df.merge(first_seen, on='contact_id', how='left')
    
    print(f"Extracted {len(contacts_df)} unique contacts")
    yield contacts_df.to_dict('records')


@dlt.resource(name="events", write_disposition="replace")
def load_events():
    """Extract all user engagement events"""
    events_df = df_clean[df_clean['Events ID'].notna()].copy()
    
    events_df['contact_id'] = events_df['Contact ID']
    if 'event_id' not in events_df.columns:
        events_df['event_id'] = events_df['Events ID']
    if 'event_category' not in events_df.columns and 'Events Category' in events_df.columns:
        events_df['event_category'] = events_df['Events Category']
    if 'Events Create date' in events_df.columns:
        events_df['event_date'] = events_df['Events Create date']
    
    print(f"Extracted {len(events_df)} events")
    yield events_df.to_dict('records')


@dlt.resource(name="utm_attribution", write_disposition="replace")
def load_utm_attribution():
    """Create attribution summary by UTM source"""
    attribution_df = df_clean.groupby('utm_source').agg({
        'Contact ID': 'nunique',
        'Events ID': lambda x: x.notna().sum()
    }).reset_index()
    
    attribution_df.columns = ['utm_source', 'unique_contacts', 'total_events']
    
    print(f"Created attribution summary for {len(attribution_df)} sources")
    yield attribution_df.to_dict('records')

print("dlt resources defined successfully")

dlt resources defined successfully


## Step 6: Execute Pipeline and Load to DuckDB

In [9]:
# Create and run dlt pipeline
pipeline = dlt.pipeline(
    pipeline_name="utm_attribution_pipeline",
    destination="duckdb",
    dataset_name=CONFIG['dataset_name']
)

load_info = pipeline.run([
    load_contacts(),
    load_events(),
    load_utm_attribution()
])

print("\nPipeline execution completed")
print(f"Database location: {CONFIG['database_name']}")
print(f"Dataset name: {CONFIG['dataset_name']}")

Extracted 2664 unique contacts
Extracted 40273 events
Created attribution summary for 6 sources

Pipeline execution completed
Database location: utm_attribution_pipeline.duckdb
Dataset name: user_attribution


## Step 7: Connect to DuckDB and Optimize Database

In [10]:
# Connect to DuckDB
conn = duckdb.connect(database=CONFIG['database_name'], read_only=False)

# Create indexes for better query performance
print("Creating database indexes...")
try:
    conn.execute("CREATE INDEX IF NOT EXISTS idx_contacts_utm ON user_attribution.contacts(utm_source)")
    conn.execute("CREATE INDEX IF NOT EXISTS idx_events_contact ON user_attribution.events(contact_id)")
    conn.execute("CREATE INDEX IF NOT EXISTS idx_events_category ON user_attribution.events(event_category)")
    print("Database indexes created successfully")
except Exception as e:
    print(f"Error creating indexes: {e}")

# Verify tables
tables = conn.execute("SHOW TABLES FROM user_attribution").fetchdf()
print(f"\nTables in user_attribution schema: {list(tables['name'])}")

Creating database indexes...
Database indexes created successfully

Tables in user_attribution schema: ['_dlt_loads', '_dlt_pipeline_state', '_dlt_version', 'contacts', 'events', 'utm_attribution']


## Step 8: Calculate Acquisition Metrics

In [11]:
acquisition_query = """
SELECT 
    utm_source,
    COUNT(DISTINCT contact_id) as total_users,
    ROUND(COUNT(DISTINCT contact_id) * 100.0 / SUM(COUNT(DISTINCT contact_id)) OVER (), 2) as pct_of_total
FROM user_attribution.contacts
GROUP BY utm_source
ORDER BY total_users DESC
"""

acquisition_metrics = conn.execute(acquisition_query).fetchdf()
print("ACQUISITION VOLUME BY SOURCE")
print(acquisition_metrics)
print(f"\nTotal users: {acquisition_metrics['total_users'].sum()}")

ACQUISITION VOLUME BY SOURCE
        utm_source  total_users  pct_of_total
0          unknown         2001         75.11
1  alexey_linkedin          403         15.13
2   dtc_newsletter          245          9.20
3   ActiveCampaign           12          0.45
4      adrian_test            2          0.08
5      chatgpt.com            1          0.04

Total users: 2664


## Step 9: Calculating Engagement Metrics

In [12]:
engagement_query = """
SELECT 
    c.utm_source,
    COUNT(DISTINCT c.contact_id) as total_users,
    COUNT(e.contact_id) as total_events,
    ROUND(CAST(COUNT(e.contact_id) AS FLOAT) / NULLIF(COUNT(DISTINCT c.contact_id), 0), 2) as avg_events_per_user,
    COUNT(DISTINCT CASE WHEN e.contact_id IS NOT NULL THEN c.contact_id END) as engaged_users,
    ROUND(CAST(COUNT(DISTINCT CASE WHEN e.contact_id IS NOT NULL THEN c.contact_id END) AS FLOAT) * 100.0 / NULLIF(COUNT(DISTINCT c.contact_id), 0), 2) as engagement_rate
FROM user_attribution.contacts c
LEFT JOIN user_attribution.events e ON c.contact_id = e.contact_id
GROUP BY c.utm_source
ORDER BY avg_events_per_user DESC
"""

engagement_metrics = conn.execute(engagement_query).fetchdf()
print("ENGAGEMENT METRICS BY SOURCE")
print(engagement_metrics)

ENGAGEMENT METRICS BY SOURCE
        utm_source  total_users  total_events  avg_events_per_user  \
0   ActiveCampaign           12           651            54.250000   
1          unknown         2001         33923            16.950001   
2  alexey_linkedin          403          4470            11.090000   
3   dtc_newsletter          245          1229             5.020000   
4      chatgpt.com            1             0             0.000000   
5      adrian_test            2             0             0.000000   

   engaged_users  engagement_rate  
0             12           100.00  
1           1951            97.50  
2            401            99.50  
3            243            99.18  
4              0             0.00  
5              0             0.00  


## Step 10: Calculating Retention Metrics

In [13]:
# Event-based retention (users with 2+ events)
retention_query = f"""
WITH user_event_counts AS (
    SELECT 
        c.utm_source,
        c.contact_id,
        COUNT(e.contact_id) as event_count
    FROM user_attribution.contacts c
    LEFT JOIN user_attribution.events e ON c.contact_id = e.contact_id
    GROUP BY c.utm_source, c.contact_id
)
SELECT 
    utm_source,
    COUNT(DISTINCT contact_id) as total_users,
    COUNT(DISTINCT CASE WHEN event_count >= {CONFIG['retention_min_events']} THEN contact_id END) as retained_users,
    ROUND(CAST(COUNT(DISTINCT CASE WHEN event_count >= {CONFIG['retention_min_events']} THEN contact_id END) AS FLOAT) * 100.0 / NULLIF(COUNT(DISTINCT contact_id), 0), 2) as retention_rate,
    ROUND(AVG(event_count), 2) as avg_events_per_user,
    MAX(event_count) as max_events_per_user
FROM user_event_counts
GROUP BY utm_source
ORDER BY retention_rate DESC
"""

retention_metrics = conn.execute(retention_query).fetchdf()
print(f"RETENTION METRICS BY SOURCE ({CONFIG['retention_min_events']}+ events)")
print(retention_metrics)

RETENTION METRICS BY SOURCE (2+ events)
        utm_source  total_users  retained_users  retention_rate  \
0   ActiveCampaign           12              12      100.000000   
1  alexey_linkedin          403             401       99.500000   
2   dtc_newsletter          245             243       99.180000   
3          unknown         2001            1833       91.599998   
4      adrian_test            2               0        0.000000   
5      chatgpt.com            1               0        0.000000   

   avg_events_per_user  max_events_per_user  
0                54.25                  125  
1                11.09                  162  
2                 5.02                  155  
3                16.95                  148  
4                 0.00                    0  
5                 0.00                    0  


In [14]:
# Time-based retention analysis
time_retention_query = f"""
WITH valid_events AS (
    SELECT *
    FROM user_attribution.events
    WHERE TRY_CAST(event_date AS TIMESTAMP) IS NOT NULL
),
first_last_activity AS (
    SELECT 
        c.utm_source,
        c.contact_id,
        MIN(TRY_CAST(e.event_date AS TIMESTAMP)) AS first_event,
        MAX(TRY_CAST(e.event_date AS TIMESTAMP)) AS last_event,
        EXTRACT(EPOCH FROM MAX(TRY_CAST(e.event_date AS TIMESTAMP)) - MIN(TRY_CAST(e.event_date AS TIMESTAMP))) / 86400.0 AS days_active
    FROM user_attribution.contacts c
    JOIN valid_events e 
        ON c.contact_id = e.contact_id
    GROUP BY c.utm_source, c.contact_id
)
SELECT 
    utm_source,
    COUNT(*) AS total_users,
    ROUND(AVG(days_active), 2) AS avg_days_active,
    ROUND(COUNT(CASE WHEN days_active >= 7 THEN 1 END) * 100.0 / NULLIF(COUNT(*), 0), 2) AS week_retention_rate,
    ROUND(COUNT(CASE WHEN days_active >= 30 THEN 1 END) * 100.0 / NULLIF(COUNT(*), 0), 2) AS month_retention_rate,
    MAX(days_active) AS max_days_active
FROM first_last_activity
GROUP BY utm_source
ORDER BY week_retention_rate DESC;
"""

try:
    time_retention_metrics = conn.execute(time_retention_query).fetchdf()
    print("TIME-BASED RETENTION METRICS BY SOURCE")
    print(time_retention_metrics)
    print(f"\nWeek retention = users active for {CONFIG['retention_min_days']}+ days")
    print(f"Month retention = users active for {CONFIG['retention_month_days']}+ days")
except Exception as e:
    print(f"Error calculating time-based retention: {e}")
    time_retention_metrics = None

TIME-BASED RETENTION METRICS BY SOURCE
        utm_source  total_users  avg_days_active  week_retention_rate  \
0          unknown         1951           174.01                92.41   
1   ActiveCampaign           12           226.07                91.67   
2  alexey_linkedin          401            85.20                41.90   
3   dtc_newsletter          243            74.77                36.63   

   month_retention_rate  max_days_active  
0                 92.36       383.103472  
1                 91.67       383.103472  
2                 41.90       383.103472  
3                 36.63       383.103472  

Week retention = users active for 7+ days
Month retention = users active for 30+ days


## Step 11: Acquisition Visualization,Cohort Analysis and Statistical Testing

In [15]:
# Acquisition volume chart
fig = px.bar(
    acquisition_metrics.sort_values('total_users', ascending=True),
    x='total_users',
    y='utm_source',
    orientation='h',
    title='User Acquisition Volume by UTM Source',
    labels={'total_users': 'Number of Users', 'utm_source': 'UTM Source'},
    text='total_users',
    color='total_users',
    color_continuous_scale='Blues'
)
fig.update_traces(textposition='outside')
fig.update_layout(height=max(400, len(acquisition_metrics) * 30), showlegend=False)
fig.show()

# Create a pie chart showing market share by UTM source
fig = px.pie(
    acquisition_metrics.sort_values('total_users', ascending=False),
    values='total_users',
    names='utm_source',
    title='UTM Source Market Share Distribution',
    color_discrete_sequence=px.colors.qualitative.Bold,
    hover_data=['pct_of_total']
)

# Improve presentation with better formatting
fig.update_traces(
    textposition='inside',
    textinfo='percent+label',
    hovertemplate='<b>%{label}</b><br>Users: %{value}<br>Market share: %{percent}<extra></extra>'
)

# Enhance layout
fig.update_layout(
    title={
        'text': 'UTM Source Market Share Distribution',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 20, 'color': '#333333'}
    },
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.2,
        xanchor="center",
        x=0.5
    ),
    height=600,
    margin=dict(t=80, b=120, l=40, r=40)
)

fig.show()

# Engagement scatter plot
fig = px.scatter(
    engagement_metrics,
    x='total_users',
    y='avg_events_per_user',
    size='total_events',
    text='utm_source',
    title='Acquisition Volume vs User Engagement',
    labels={
        'total_users': 'Total Users Acquired',
        'avg_events_per_user': 'Avg Events Per User',
        'total_events': 'Total Events'
    },
    color='avg_events_per_user',
    color_continuous_scale='Viridis'
)
fig.update_traces(textposition='top center')
fig.update_layout(height=600)
fig.show()

In [16]:
# Cohort analysis by signup month
cohort_query = f"""
WITH contact_cohorts AS (
    SELECT 
        c.contact_id,
        c.utm_source,
        DATE_TRUNC('month', c.first_seen_at) as cohort_month
    FROM user_attribution.contacts c
    WHERE c.first_seen_at IS NOT NULL
),
cohort_events AS (
    SELECT 
        cc.cohort_month,
        cc.utm_source,
        COUNT(DISTINCT cc.contact_id) as cohort_size,
        COUNT(e.contact_id) as total_events,
        ROUND(CAST(COUNT(e.contact_id) AS FLOAT) / NULLIF(COUNT(DISTINCT cc.contact_id), 0), 2) as avg_events_per_user
    FROM contact_cohorts cc
    LEFT JOIN user_attribution.events e ON cc.contact_id = e.contact_id
    GROUP BY cc.cohort_month, cc.utm_source
)
SELECT 
    cohort_month,
    utm_source,
    cohort_size,
    total_events,
    avg_events_per_user
FROM cohort_events
WHERE cohort_size >= {CONFIG['min_users_for_analysis']}
ORDER BY cohort_month DESC, avg_events_per_user DESC
LIMIT 50
"""
try:
    cohort_metrics = conn.execute(cohort_query).fetchdf()
    if len(cohort_metrics) > 0:
        print("COHORT ANALYSIS (Monthly)")
        print(cohort_metrics.head(20))
        
        if len(cohort_metrics) > 0:
            color_palette = px.colors.qualitative.Plotly
            fig = px.line(
                cohort_metrics,
                x='cohort_month',
                y='avg_events_per_user',
                color='utm_source',
                title='User Engagement by Signup Cohort',
                labels={
                    'cohort_month': 'Signup Month',
                    'avg_events_per_user': 'Avg Events Per User'
                },
                markers=True,
                color_discrete_sequence=color_palette
            )
            fig.update_layout(height=500)
            fig.show()
    else:
        print("No cohort data available (insufficient date information)")
except Exception as e:
    print(f"Error calculating cohorts: {e}")

COHORT ANALYSIS (Monthly)
                cohort_month       utm_source  cohort_size  total_events  \
0  2025-09-01 00:00:00+02:00          unknown           58            78   
1  2025-08-01 00:00:00+02:00          unknown           31           172   
2  2025-07-01 00:00:00+02:00  alexey_linkedin           45           153   
3  2025-07-01 00:00:00+02:00   dtc_newsletter           54           174   
4  2025-07-01 00:00:00+02:00          unknown           69           192   
5  2025-06-01 00:00:00+02:00   dtc_newsletter          179           726   
6  2025-06-01 00:00:00+02:00          unknown          106           382   
7  2025-06-01 00:00:00+02:00  alexey_linkedin          295           945   
8  2025-05-01 00:00:00+02:00          unknown           61           699   
9  2025-04-01 00:00:00+02:00          unknown           67          1906   
10 2025-03-01 00:00:00+01:00  alexey_linkedin            6           303   
11 2025-03-01 00:00:00+01:00          unknown          106    

In [17]:
def visualize_cohort_retention(conn, config):
    """
    Visualize retention rates by week and month across different cohorts and UTM sources
    
    Args:
        conn: DuckDB connection
        config: Configuration dictionary with parameters
    """
    # First creating a clean events view to handle invalid dates
    clean_events_query = """
    CREATE OR REPLACE TEMPORARY VIEW clean_events AS
    SELECT 
        contact_id,
        event_id,
        event_category,
        CASE 
            WHEN TRY_CAST(event_date AS DATE) IS NOT NULL THEN TRY_CAST(event_date AS DATE)
            ELSE NULL
        END AS clean_date
    FROM user_attribution.events
    WHERE event_date IS NOT NULL
    """
    
    # SQL query for cohort retention analysis using clean dates
    cohort_retention_query = """
    WITH user_first_activity AS (
        -- Get each user's first activity date
        SELECT
            c.contact_id,
            c.utm_source,
            MIN(e.clean_date) AS first_date
        FROM user_attribution.contacts c
        JOIN clean_events e ON c.contact_id = e.contact_id
        WHERE e.clean_date IS NOT NULL
        GROUP BY c.contact_id, c.utm_source
    ),
    
    user_activity_weeks AS (
        -- Get each subsequent week of activity for each user
        SELECT
            u.contact_id,
            u.utm_source,
            u.first_date,
            e.clean_date AS activity_date,
            FLOOR(DATEDIFF('day', u.first_date, e.clean_date) / 7) AS week_number
        FROM user_first_activity u
        JOIN clean_events e ON u.contact_id = e.contact_id
        WHERE e.clean_date IS NOT NULL
        AND DATEDIFF('day', u.first_date, e.clean_date) >= 0
        AND DATEDIFF('day', u.first_date, e.clean_date) <= 84 -- Track up to 12 weeks
    ),
    
    user_activity_months AS (
        -- Get each subsequent month of activity for each user
        SELECT
            u.contact_id,
            u.utm_source,
            u.first_date,
            e.clean_date AS activity_date,
            FLOOR(DATEDIFF('day', u.first_date, e.clean_date) / 30) AS month_number
        FROM user_first_activity u
        JOIN clean_events e ON u.contact_id = e.contact_id
        WHERE e.clean_date IS NOT NULL
        AND DATEDIFF('day', u.first_date, e.clean_date) >= 0
        AND DATEDIFF('day', u.first_date, e.clean_date) <= 180 -- Track up to 6 months
    ),
    
    weekly_retention AS (
        -- Calculate weekly retention rates
        SELECT
            utm_source,
            week_number,
            COUNT(DISTINCT contact_id) AS active_users,
            (SELECT COUNT(DISTINCT contact_id) FROM user_first_activity WHERE utm_source = w.utm_source) AS total_users,
            ROUND(COUNT(DISTINCT contact_id) * 100.0 / 
                NULLIF((SELECT COUNT(DISTINCT contact_id) FROM user_first_activity WHERE utm_source = w.utm_source), 0), 1) AS retention_rate
        FROM user_activity_weeks w
        GROUP BY utm_source, week_number
    ),
    
    monthly_retention AS (
        -- Calculate monthly retention rates
        SELECT
            utm_source,
            month_number,
            COUNT(DISTINCT contact_id) AS active_users,
            (SELECT COUNT(DISTINCT contact_id) FROM user_first_activity WHERE utm_source = m.utm_source) AS total_users,
            ROUND(COUNT(DISTINCT contact_id) * 100.0 / 
                NULLIF((SELECT COUNT(DISTINCT contact_id) FROM user_first_activity WHERE utm_source = m.utm_source), 0), 1) AS retention_rate
        FROM user_activity_months m
        GROUP BY utm_source, month_number
    )
    
    -- Select weekly retention
    SELECT 
        'weekly' AS retention_type,
        utm_source,
        week_number AS period_number,
        active_users,
        total_users,
        retention_rate
    FROM weekly_retention
    WHERE total_users >= 5  -- Minimum sample size
    
    UNION ALL
    
    -- Select monthly retention
    SELECT 
        'monthly' AS retention_type,
        utm_source,
        month_number AS period_number,
        active_users,
        total_users,
        retention_rate
    FROM monthly_retention
    WHERE total_users >= 5  -- Minimum sample size
    
    ORDER BY retention_type, utm_source, period_number
    """
    
    try:
        conn.execute(clean_events_query)
        
        # Executing the query
        retention_data = conn.execute(cohort_retention_query).fetchdf()
        
        if len(retention_data) == 0:
            print("No sufficient retention data available for cohort analysis.")
            return
        
        # Split into weekly and monthly data
        weekly_data = retention_data[retention_data['retention_type'] == 'weekly']
        monthly_data = retention_data[retention_data['retention_type'] == 'monthly']
        
        # 1. Weekly retention heatmap
        if len(weekly_data) > 0:
            # Pivot the data
            weekly_pivot = weekly_data.pivot(
                index='utm_source', 
                columns='period_number', 
                values='retention_rate'
            ).fillna(0)
            
            # Renaming columns for better readability
            weekly_pivot.columns = [f'Week {int(col)+1}' for col in weekly_pivot.columns]
            
            # Creating a heatmap
            fig = px.imshow(
                weekly_pivot,
                labels=dict(x="Weeks Since First Activity", y="UTM Source", color="Retention Rate (%)"),
                x=weekly_pivot.columns,
                y=weekly_pivot.index,
                color_continuous_scale='RdYlGn',
                text_auto='.1f',
                aspect='auto',
                title='Weekly Retention by UTM Source'
            )
            fig.update_layout(height=max(400, len(weekly_pivot) * 40))
            fig.show()
            
            # Line chart for weekly retention
            fig = go.Figure()

            color_palette = px.colors.qualitative.Plotly  
            utm_sources = monthly_data['utm_source'].unique()
            color_map = {source: color_palette[i % len(color_palette)] for i, source in enumerate(utm_sources)}
            
            for source in weekly_data['utm_source'].unique():
                source_data = weekly_data[weekly_data['utm_source'] == source]
                fig.add_trace(go.Scatter(
                    x=source_data['period_number'] + 1,  # +1 for readability (Week 1, 2, etc.)
                    y=source_data['retention_rate'],
                    mode='lines+markers',
                    name=source,
                    line=dict(color=color_map[source]),
                    marker=dict(color=color_map[source]),
                    text=source_data['retention_rate'].apply(lambda x: f"{x:.1f}%"),
                    hovertemplate='%{text}<extra></extra>'
                ))
            
            fig.update_layout(
                title='Weekly Retention Curves by UTM Source',
                xaxis_title='Weeks Since First Activity',
                yaxis_title='Retention Rate (%)',
                xaxis=dict(tickmode='linear', tick0=1, dtick=1),
                legend=dict(
                    orientation="h",
                    yanchor="bottom",
                    y=1.02,
                    xanchor="right",
                    x=1
                ),
                height=500
            )
            fig.show()
        
        # 2. Monthly retention heatmap
        if len(monthly_data) > 0:
            # Pivot the data
            monthly_pivot = monthly_data.pivot(
                index='utm_source', 
                columns='period_number', 
                values='retention_rate'
            ).fillna(0)
            
            # Rename columns for better readability
            monthly_pivot.columns = [f'Month {int(col)+1}' for col in monthly_pivot.columns]
            
            # Create heatmap
            fig = px.imshow(
                monthly_pivot,
                labels=dict(x="Months Since First Activity", y="UTM Source", color="Retention Rate (%)"),
                x=monthly_pivot.columns,
                y=monthly_pivot.index,
                color_continuous_scale='RdYlGn',
                text_auto='.1f',
                aspect='auto',
                title='Monthly Retention by UTM Source'
            )
            fig.update_layout(height=max(400, len(monthly_pivot) * 40))
            fig.show()
            
            # Line chart for monthly retention
            fig = go.Figure()
            
            for source in monthly_data['utm_source'].unique():
                source_data = monthly_data[monthly_data['utm_source'] == source]
                fig.add_trace(go.Scatter(
                    x=source_data['period_number'] + 1,  # +1 for readability (Month 1, 2, etc.)
                    y=source_data['retention_rate'],
                    mode='lines+markers',
                    name=source,
                    line=dict(color=color_map[source]),
                    marker=dict(color=color_map[source]),
                    text=source_data['retention_rate'].apply(lambda x: f"{x:.1f}%"),
                    hovertemplate='%{text}<extra></extra>'
                ))
            
            fig.update_layout(
                title='Monthly Retention Curves by UTM Source',
                xaxis_title='Months Since First Activity',
                yaxis_title='Retention Rate (%)',
                xaxis=dict(tickmode='linear', tick0=1, dtick=1),
                legend=dict(
                    orientation="h",
                    yanchor="bottom",
                    y=1.02,
                    xanchor="right",
                    x=1
                ),
                height=500
            )
            fig.show()
                    
    except Exception as e:
        print(f"Error generating cohort retention visualizations: {e}")
        import traceback
        traceback.print_exc()

# Call the function
visualize_cohort_retention(conn, CONFIG)

In [18]:
# Query to get UTM source acquisition over time
utm_trends_query = """
SELECT 
    DATE_TRUNC('month', c.first_seen_at) as month,
    c.utm_source,
    COUNT(DISTINCT c.contact_id) as new_users
FROM user_attribution.contacts c
WHERE c.first_seen_at IS NOT NULL
GROUP BY DATE_TRUNC('month', c.first_seen_at), c.utm_source
ORDER BY month, utm_source
"""

try:
    utm_trends = conn.execute(utm_trends_query).fetchdf()
    
    # Filter to sources with meaningful data (optional)
    if len(utm_trends) > 0:
        # Print the data
        print("UTM SOURCE ACQUISITION TRENDS")
        print(utm_trends.head(10))
        
        # Line chart showing the same data
        fig = px.line(
            utm_trends,
            x='month',
            y='new_users',
            color='utm_source',
            title='UTM Source Acquisition Trends Over Time',
            labels={
                'month': 'Month',
                'new_users': 'New Users Acquired',
                'utm_source': 'UTM Source'
            },
            markers=True,
            color_discrete_sequence=px.colors.qualitative.Bold,
        )
        fig.update_layout(height=600)
        fig.show()

        # Stacked bar chart, clearer for monthly data
        fig = px.bar(
            utm_trends,
            x='month',
            y='new_users',
            color='utm_source',
            title='Monthly User Acquisition by UTM Source',
            labels={
                'month': 'Month',
                'new_users': 'New Users Acquired',
                'utm_source': 'UTM Source'
            },
            color_discrete_sequence=px.colors.qualitative.Bold,
        )
        fig.update_layout(height=600, bargap=0.1)
        fig.show()
except Exception as e:
    print(f"Error generating UTM trends visualization: {e}")

UTM SOURCE ACQUISITION TRENDS
                      month       utm_source  new_users
0 2024-08-01 00:00:00+02:00  alexey_linkedin          2
1 2024-08-01 00:00:00+02:00          unknown         83
2 2024-09-01 00:00:00+02:00   ActiveCampaign          2
3 2024-09-01 00:00:00+02:00  alexey_linkedin          8
4 2024-09-01 00:00:00+02:00   dtc_newsletter          2
5 2024-09-01 00:00:00+02:00          unknown        339
6 2024-10-01 00:00:00+02:00   ActiveCampaign          3
7 2024-10-01 00:00:00+02:00      adrian_test          1
8 2024-10-01 00:00:00+02:00  alexey_linkedin         11
9 2024-10-01 00:00:00+02:00   dtc_newsletter          1


### The t-test tells you if two groups are different.
### The p-value tells you how likely it is that the difference is due to random chance.
T-statistic: The high t-value (4.448) indicates a large standardized difference between the groups. The gap between ActiveCampaign's average (54.25 events) and unknown sources' average (16.95 events) is substantial and meaningful.
P-Value: An extremely small p-value (<0.0001) indicates the difference is almost certainly not due to random chance. We can be very confident (>99.99%) that ActiveCampaign users genuinely engage more with the product than users from unknown sources.

This test provides strong statistical evidence that ActiveCampaign is a significantly higher-quality acquisition source that drives much better user engagement.

Similarly other sources can be tested statistically.

In [19]:
# Statistical significance testing (t-test between top 2 sources)
utm_sources = engagement_metrics['utm_source'].tolist()

if len(utm_sources) >= 2:
    source1, source2 = utm_sources[0], utm_sources[1]
    
    query1 = f"SELECT COUNT(e.contact_id) as event_count FROM user_attribution.contacts c LEFT JOIN user_attribution.events e ON c.contact_id = e.contact_id WHERE c.utm_source = '{source1}' GROUP BY c.contact_id"
    query2 = f"SELECT COUNT(e.contact_id) as event_count FROM user_attribution.contacts c LEFT JOIN user_attribution.events e ON c.contact_id = e.contact_id WHERE c.utm_source = '{source2}' GROUP BY c.contact_id"
    
    try:
        events1 = conn.execute(query1).fetchdf()['event_count'].values
        events2 = conn.execute(query2).fetchdf()['event_count'].values
        
        t_stat, p_value = stats.ttest_ind(events1, events2)
        
        print("\nSTATISTICAL SIGNIFICANCE TEST")
        print(f"Comparing: '{source1}' vs '{source2}'")
        print(f"  {source1}: Mean events = {events1.mean():.2f}, N = {len(events1)}")
        print(f"  {source2}: Mean events = {events2.mean():.2f}, N = {len(events2)}")
        print(f"  T-statistic: {t_stat:.4f}")
        print(f"  P-value: {p_value:.4f}")
        
        if p_value < 0.05:
            print(f"  Result: SIGNIFICANT (p < 0.05)")
        else:
            print(f"  Result: NOT SIGNIFICANT (p >= 0.05)")
    except Exception as e:
        print(f"Error performing statistical test: {e}")
else:
    print("Not enough sources for comparison")


STATISTICAL SIGNIFICANCE TEST
Comparing: 'ActiveCampaign' vs 'unknown'
  ActiveCampaign: Mean events = 54.25, N = 12
  unknown: Mean events = 16.95, N = 2001
  T-statistic: 5.3380
  P-value: 0.0000
  Result: SIGNIFICANT (p < 0.05)


## Step 12: Calculate Composite Quality Scores

In [20]:
# Merging all metrics
quality_df = acquisition_metrics[['utm_source', 'total_users']].merge(
    engagement_metrics[['utm_source', 'avg_events_per_user', 'engagement_rate']],
    on='utm_source'
).merge(
    retention_metrics[['utm_source', 'retention_rate']],
    on='utm_source'
)

# Normalizing metrics to 0-100 scale
scaler = MinMaxScaler(feature_range=(0, 100))
quality_df['volume_score'] = scaler.fit_transform(quality_df[['total_users']])
quality_df['engagement_score'] = scaler.fit_transform(quality_df[['avg_events_per_user']])
quality_df['retention_score'] = quality_df['retention_rate']  

# Calculating composite quality score with configured weights
quality_df['quality_score'] = (
    CONFIG['volume_weight'] * quality_df['volume_score'] +
    CONFIG['engagement_weight'] * quality_df['engagement_score'] +
    CONFIG['retention_weight'] * quality_df['retention_score']
).round(2)

# Ranking the sources
quality_df = quality_df.sort_values('quality_score', ascending=False)
quality_df['rank'] = range(1, len(quality_df) + 1)

print("SOURCE QUALITY RANKING")
print(quality_df[['rank', 'utm_source', 'quality_score', 'total_users', 'avg_events_per_user', 'retention_rate']])

SOURCE QUALITY RANKING
   rank       utm_source  quality_score  total_users  avg_events_per_user  \
3     1   ActiveCampaign          80.11           12            54.250000   
0     2          unknown          69.14         2001            16.950001   
1     3  alexey_linkedin          52.00          403            11.090000   
2     4   dtc_newsletter          45.81          245             5.020000   
4     5      adrian_test           0.01            2             0.000000   
5     6      chatgpt.com           0.00            1             0.000000   

   retention_rate  
3      100.000000  
0       91.599998  
1       99.500000  
2       99.180000  
4        0.000000  
5        0.000000  


## Step 13: Segment Sources into Quality Tiers

In [21]:
def categorize_quality(score):
    """Categorize quality score into tiers"""
    if score >= CONFIG['high_quality_threshold']:
        return 'High Quality'
    elif score >= CONFIG['medium_quality_threshold']:
        return 'Medium Quality'
    else:
        return 'Low Quality'

quality_df['quality_tier'] = quality_df['quality_score'].apply(categorize_quality)

# Summary by tier
tier_summary = quality_df.groupby('quality_tier').agg({
    'utm_source': 'count',
    'total_users': 'sum',
    'quality_score': 'mean'
}).round(2)
tier_summary.columns = ['num_sources', 'total_users', 'avg_quality_score']

print("QUALITY TIER SUMMARY")
print(tier_summary)

high_quality = quality_df[quality_df['quality_tier'] == 'High Quality']
medium_quality = quality_df[quality_df['quality_tier'] == 'Medium Quality']
low_quality = quality_df[quality_df['quality_tier'] == 'Low Quality']

print(f"\nHigh Quality sources: {len(high_quality)}")
print(f"Medium Quality sources: {len(medium_quality)}")
print(f"Low Quality sources: {len(low_quality)}")

QUALITY TIER SUMMARY
                num_sources  total_users  avg_quality_score
quality_tier                                               
High Quality              1           12              80.11
Low Quality               2            3               0.00
Medium Quality            3         2649              55.65

High Quality sources: 1
Medium Quality sources: 3
Low Quality sources: 2


In [22]:
# Quality score visualization
fig = px.bar(
    quality_df.sort_values('quality_score', ascending=True),
    x='quality_score',
    y='utm_source',
    orientation='h',
    title='Composite Quality Score by UTM Source',
    labels={'quality_score': 'Quality Score (0-100)', 'utm_source': 'UTM Source'},
    text='quality_score',
    color='quality_score',
    color_continuous_scale='RdYlGn'
)
fig.update_traces(textposition='outside')
fig.update_layout(height=max(400, len(quality_df) * 30), showlegend=False)
fig.show()



# Quality tier segmentation
tier_palette = {
    'High Quality': '#2ca02c',    # green
    'Medium Quality':'#ff7f0e',  # orange
    'Low Quality': '#d62728'      # red
}
fig = px.scatter(
    quality_df,
    x='total_users',
    y='quality_score',
    size='avg_events_per_user',
    color='quality_tier',
    text='utm_source',
    title='UTM Source Segmentation: Volume vs Quality',
    labels={
        'total_users': 'Total Users Acquired',
        'quality_score': 'Quality Score',
        'quality_tier': 'Quality Tier'
    },
    color_discrete_map=tier_palette
)
fig.update_traces(textposition='top center')
fig.update_layout(height=600)
fig.show()

## Step 14: Generating Business Insights

In [23]:
# Business Insight: Specific Source Comparisons

print("=" * 100)
print("📊 BUSINESS INSIGHTS")
print("=" * 100)

# Creating a comparison dataset for specific insights
insight_sources = quality_df[quality_df['utm_source'].isin(['alexey_linkedin', 'dtc_newsletter', 'ActiveCampaign', 'unknown', 'chatgpt.com'])].copy()

if len(insight_sources) > 0:
    # 1. Comparison Chart: High Engagement vs High Traffic
    fig = go.Figure()
    
    # Add bars for user volume
    fig.add_trace(go.Bar(
        name='User Volume',
        x=insight_sources['utm_source'],
        y=insight_sources['total_users'],
        marker_color='lightblue',
        text=insight_sources['total_users'],
        textposition='auto',
        yaxis='y'
    ))
    
    # Add line for engagement
    fig.add_trace(go.Scatter(
        name='Avg Events per User',
        x=insight_sources['utm_source'],
        y=insight_sources['avg_events_per_user'],
        mode='lines+markers+text',
        marker=dict(size=12, color='crimson'),
        line=dict(width=3, color='crimson'),
        text=insight_sources['avg_events_per_user'].round(1),
        textposition='top center',
        yaxis='y2'
    ))
    
    fig.update_layout(
        title='📈 Traffic vs Engagement',
        xaxis=dict(title='UTM Source'),
        yaxis=dict(
            title='Number of Users Acquired',
            titlefont=dict(color='lightblue'),
            tickfont=dict(color='lightblue')
        ),
        yaxis2=dict(
            title='Avg Events per User (Engagement)',
            titlefont=dict(color='crimson'),
            tickfont=dict(color='crimson'),
            overlaying='y',
            side='right'
        ),
        height=500,
        hovermode='x unified',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    fig.show()
    
    # 2. Radar Chart: Multi-dimensional Quality Comparison
    if 'alexey_linkedin' in insight_sources['utm_source'].values and 'dtc_newsletter' in insight_sources['utm_source'].values:
        
        alexey_data = insight_sources[insight_sources['utm_source'] == 'alexey_linkedin'].iloc[0]
        chatgpt_data = insight_sources[insight_sources['utm_source'] == 'dtc_newsletter'].iloc[0]

        categories = ['Volume Score', 'Engagement Score', 'Retention Score', 'Quality Score']
        
        fig = go.Figure()
        
        fig.add_trace(go.Scatterpolar(
            r=[
                alexey_data['volume_score'],
                alexey_data['engagement_score'],
                alexey_data['retention_score'],
                alexey_data['quality_score']
            ],
            theta=categories,
            fill='toself',
            name='alexey_linkedin',
            line=dict(color='green', width=2),
            marker=dict(size=8)
        ))
        
        fig.add_trace(go.Scatterpolar(
            r=[
                chatgpt_data['volume_score'],
                chatgpt_data['engagement_score'],
                chatgpt_data['retention_score'],
                chatgpt_data['quality_score']
            ],
            theta=categories,
            fill='toself',
            name='dtc_newsletter',
            line=dict(color='red', width=2),
            marker=dict(size=8)
        ))
        
        fig.update_layout(
            polar=dict(
                radialaxis=dict(
                    visible=True,
                    range=[0, 100]
                )
            ),
            title='🎯 Quality Profile Comparison: LinkedIn vs dtc_newsletter',
            showlegend=True,
            height=500
        )
        
        fig.show()
    
    # 3. Funnel Chart: Conversion Efficiency
    # Create a funnel showing volume -> engagement -> retention
    funnel_data = []
            
    for _, row in insight_sources.iterrows():
        source = row['utm_source']
        total_users = row['total_users']
        engaged_users = int(total_users * row['engagement_rate'] / 100)
        retained_users = int(total_users * row['retention_rate'] / 100)
        
        funnel_data.append({
            'source': source,
            'stage': 'Acquired',
            'value': total_users,
            'color': '#1f77b4'
        })
        funnel_data.append({
            'source': source,
            'stage': 'Engaged',
            'value': engaged_users,
            'color': '#ff7f0e'
        })
        funnel_data.append({
            'source': source,
            'stage': 'Retained',
            'value': retained_users,
            'color': '#2ca02c'
        })
    
    funnel_df = pd.DataFrame(funnel_data)
    
    # Create subplots for each source
    from plotly.subplots import make_subplots
    
    sources = insight_sources['utm_source'].tolist()
    fig = make_subplots(
        rows=1, 
        cols=len(sources),
        subplot_titles=sources,
        specs=[[{'type': 'funnel'}] * len(sources)]
    )
    
    for i, source in enumerate(sources, 1):
        source_data = funnel_df[funnel_df['source'] == source]
        
        fig.add_trace(
            go.Funnel(
                y=source_data['stage'],
                x=source_data['value'],
                textinfo='value+percent initial',
                marker=dict(
                    color=['#1f77b4', '#ff7f0e', '#2ca02c']
                )
            ),
            row=1, col=i
        )
    
    fig.update_layout(
        title_text='🔄 User Journey Funnel: Acquisition → Engagement → Retention',
        height=500,
        showlegend=False
    )
    
    fig.show()
    
    # 4. Key Insights Summary Table
    print("\n" + "=" * 100)
    print("💡 KEY INSIGHTS SUMMARY")
    print("=" * 100)
    
    for _, row in insight_sources.iterrows():
        source = row['utm_source']
        
        print(f"\n📍 {source.upper()}")
        print(f"   Quality Tier: {row['quality_tier']}")
        print(f"   Quality Score: {row['quality_score']}/100")
        print(f"   Users Acquired: {int(row['total_users']):,}")
        print(f"   Avg Events/User: {row['avg_events_per_user']:.1f}")
        print(f"   Retention Rate: {row['retention_rate']:.1f}%")
        
        # Generate insight
        if source == 'alexey_linkedin':
            print(f"   💡 INSIGHT: Alexey's LinkedIn posts lead to the highest engaged users.")
            print(f"      → Users from this source generate {row['avg_events_per_user']:.1f} events on average")
            print(f"      → {row['retention_rate']:.1f}% retention rate shows strong user quality")
            print(f"      → RECOMMENDATION: Increase content frequency and experiment with LinkedIn ads")
        
        elif source == 'chatgpt.com':
            print(f"   ⚠️  INSIGHT: ChatGPT links drove only one user and no engagement. Can be considered noise")
            print(f"      → {row['retention_rate']:.1f}% retention indicates poor user-product fit")
            print(f"      → RECOMMENDATION: Reduce spend or implement better pre-qualification")

        elif source == 'dtc_newsletter':
            print(f"   💡 INSIGHT: DTC Newsletter drives consistent engagement.")
            print(f"      → Users from this source generate {row['avg_events_per_user']:.1f} events on average")
            print(f"      → {row['retention_rate']:.1f}% retention rate shows strong user quality")
            print(f"      → RECOMMENDATION: Continue leveraging newsletter content and explore partnerships")

        elif source == 'ActiveCampaign':
            print(f"   🌟 INSIGHT: Top-performing source across all metrics.")
            print(f"      → Highest quality score ({row['quality_score']}/100)")
            print(f"      → Strong engagement ({row['avg_events_per_user']:.1f} events) + volume ({int(row['total_users'])} users)")
            print(f"      → RECOMMENDATION: Increase budget by 25-30%")
        
        elif source == 'unknown':
            print(f"   ❓ INSIGHT: Unknown sources contribute significant volume but quality varies.")
            print(f"      → Implement better UTM tracking to identify actual sources")
            print(f"      → RECOMMENDATION: Audit marketing campaigns for missing UTM parameters")
    
    print("\n" + "=" * 100)

else:
    print("Insufficient data for specific source comparisons")

print("\n" + "=" * 100)

📊 BUSINESS INSIGHTS



💡 KEY INSIGHTS SUMMARY

📍 ACTIVECAMPAIGN
   Quality Tier: High Quality
   Quality Score: 80.11/100
   Users Acquired: 12
   Avg Events/User: 54.2
   Retention Rate: 100.0%
   🌟 INSIGHT: Top-performing source across all metrics.
      → Highest quality score (80.11/100)
      → Strong engagement (54.2 events) + volume (12 users)
      → RECOMMENDATION: Increase budget by 25-30%

📍 UNKNOWN
   Quality Tier: Medium Quality
   Quality Score: 69.14/100
   Users Acquired: 2,001
   Avg Events/User: 17.0
   Retention Rate: 91.6%
   ❓ INSIGHT: Unknown sources contribute significant volume but quality varies.
      → Implement better UTM tracking to identify actual sources
      → RECOMMENDATION: Audit marketing campaigns for missing UTM parameters

📍 ALEXEY_LINKEDIN
   Quality Tier: Medium Quality
   Quality Score: 52.0/100
   Users Acquired: 403
   Avg Events/User: 11.1
   Retention Rate: 99.5%
   💡 INSIGHT: Alexey's LinkedIn posts lead to the highest engaged users.
      → Users from this sou

## Executive Summary

This analysis evaluated the quality of acquisition sources (`utm_source`) using three metrics:

- **Volume (20%)** – Number of users acquired  
- **Engagement (40%)** – Average events per user  
- **Retention (40%)** – Percentage of users who remain active  

Sources were grouped into three tiers requiring different strategic actions.

---

## Recommendations by Quality Tier

### High-Quality Sources (Score ≥ 70) ActiveCampaign
**Strategy:** Invest More  
- Increase marketing budget by 20–30%.  
- Expand campaigns and targeting.  
- Use for high-value offerings and product launches.  
- Build lookalike audiences and apply advanced attribution models.  
- Continue A/B testing to refine messaging.  

---

### Medium-Quality Sources (Score 40–69) alexey_linkedin, dtc_newsletter

- Analyze user drop-off points.  
- Create tailored onboarding for each source.  
- Deliver core value faster to new users.  
- Run re-engagement and feature education campaigns.  
- Track 7/14/30-day engagement milestones.  


---

### Low-Quality Sources (Score < 40) chatgpt.com, adrian_test
**Strategy:** Reduce Spend or Retarget  
- Cut or pause underperforming campaigns.  
- Refine audience targeting and messaging.  
- Reassess channel fit with target customers.  
- Compare acquisition cost with lifetime value.  
- Test new landing pages or signup flows.  
---

## Step 15: Export Results

In [24]:
# Export detailed results
output_df = quality_df[[
    'rank', 'utm_source', 'quality_score',
    'total_users', 'avg_events_per_user', 'retention_rate',
    'volume_score', 'engagement_score', 'retention_score'
]].copy()

output_df.to_csv(CONFIG['output_quality_file'], index=False)
print(f"Results exported to: {CONFIG['output_quality_file']}")

# Creating summary report
summary_report = {
    'Analysis Date': datetime.now().strftime('%Y-%m-%d'),
    'Total Sources Analyzed': len(quality_df),
    'Total Users': int(quality_df['total_users'].sum()),
    'High Quality Sources': len(high_quality),
    'Medium Quality Sources': len(medium_quality),
    'Low Quality Sources': len(low_quality),
    'Best Source': quality_df.iloc[0]['utm_source'],
    'Best Source Quality Score': quality_df.iloc[0]['quality_score'],
    'Avg Quality Score': round(quality_df['quality_score'].mean(), 2)
}

summary_df = pd.DataFrame([summary_report])
summary_df.to_csv(CONFIG['output_summary_file'], index=False)
print(f"Summary exported to: {CONFIG['output_summary_file']}")

print("\nAnalysis complete!")

Results exported to: utm_source_quality_analysis.csv
Summary exported to: utm_attribution_summary.csv

Analysis complete!
